# Ontologia powiązań rodzinnych

Przykład z **AI for Beginners**

W tym przykładzie weźmiemy na warsztat ontologię powiązań rodzinnych oraz prawdziwe drzewo genealogiczne, i zobaczymy jak można wykonać automatyczne wnioskowanie by znaleźć wszystkich krewnych.

### Pobieranie drzewa geneealogicznego

Jako przykład przyjmiemy drzewo genealogiczne [carskiej rodziny Romanowów](https://pl.wikipedia.org/wiki/Romanowowie). Najbardziej popularny format do opisu powiązań rodzinnych to [GEDCOM](https://pl.wikipedia.org/wiki/GEDCOM). Pobierzmy drzewo rodziny Romanowów w formacie GEDCOM.

By użyć pliku GEDCOM, wykorzystamy bibliotekę `python-gedcom`:

In [1]:
import sys
!{sys.executable} -m pip install python-gedcom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Sparsujmy plik i pokażmy całą listę osobistości:

In [3]:
from gedcom.parser import Parser
from gedcom.element.individual import IndividualElement
from gedcom.element.family import FamilyElement
g = Parser()
g.parse_file('tsars.ged')

FileNotFoundError: ignored

In [ ]:
d = g.get_element_dictionary()
[ (k,v.get_name()) for k,v in d.items() if isinstance(v,IndividualElement)] # wyświetlmy wszystkie osobistości

W poniższy sposób możemy uzyskać informację o rodzinach. Zwróci nam to **identifikatory** (musimy je skonwertować na imiona jeśli chcemy by było to bardziej czytelne)

In [ ]:
d = g.get_element_dictionary()
[ (k,[x.get_value() for x in v.get_child_elements()]) for k,v in d.items() if isinstance(v,FamilyElement)]

### Ontologia rodziny

Następnie spójrzmy na [ontologię rodziny](https://raw.githubusercontent.com/blokhin/genealogical-trees/master/data/header.ttl) zdefiniowaną jako trójki (triplets) w Semantic Webie. Ontologia ta definiuje takie powiązania jak `isUncleOf`, `isCousinOf`, i wiele innych. Wszystkie te powiązania są zdefiniowane w kontekście bazowych predykatów `isMotherOf`, `isFatherOf`, `isBrotherOf` i `isSisterOf`. Użyjemy automatycznego wnioskowania by wydedukować wszystkie inne powiązania w tej ontologii.

Poniżej mamy definicję właściwości `isAuntOf`, która jest zdefiniowana jako kompozycja `isSisterOf` i `isParentOf` (*Jako że ciocia to siostra któregoś z rodziców*).

```
fhkb:isAuntOf a owl:ObjectProperty ;
    rdfs:domain fhkb:Woman ;
    rdfs:range fhkb:Person ;
    owl:propertyChainAxiom ( fhkb:isSisterOf fhkb:isParentOf ) .
```

### Przygotowanie ontologii do wnioskowania

Dla ułatwienia, stworzymy jeden plik z ontologią który będzie zawierał oryginalne reguły z ontologii rodziny oraz fakty o osobistościach z naszego pliku GEDCOM. Przejdziemy przez plik GEDCOM wyekstrahujemy informacje o rodzinach i osobistościach i skonwertujemy je do trójek.

In [ ]:
gedcom_dict = g.get_element_dictionary()
individuals, marriages = {}, {} # nowe słowniki osób i małżeństw

# funkcja konwertująca pobrany termin do identyfikatora
def term2id(el):
    return "i" + el.get_pointer().replace('@', '').lower()

# otwieramy plik
out = open("onto.ttl","a")

# główna pętla przechodząca przez plik
for k, v in gedcom_dict.items(): # dla każdego elementu słownika
    if isinstance(v,IndividualElement): # jeśli wartość elementu to osoba
        children, siblings = set(), set() # utwórz dwa zbiory: dzieci i rodzeństwa
        idx = term2id(v)

        title = v.get_name()[0] + " " + v.get_name()[1]
        title = title.replace('"', '').replace('[', '').replace(']', '').replace('(', '').replace(')', '').strip()

        own_families = g.get_families(v, 'FAMS') # pobierz informacje o rodzinach posiadanych
        for fam in own_families: # dla każdej rodziny
            children |= set(term2id(i) for i in g.get_family_members(fam, "CHIL")) # dodaj dzieci

        parent_families = g.get_families(v, 'FAMC') # pobierz informacje o rodzinach z których się pochodzi
        if len(parent_families): # jeśli jest o takiej rodzinie informacja (zauważ, że nie uwzględniona jest tu adopcja)
            for member in g.get_family_members(parent_families[0], "CHIL"):  # każdą osobę która jest dzieckiem
                if member.get_pointer() == v.get_pointer(): # sprawdź czy nie jest tą osobą którą analizujemy
                    continue
                siblings.add(term2id(member)) # dodaj jako rodzeństwo

        #dodanie pełnej informacji o osobie do nowego słownika
        if idx in individuals:
            children |= individuals[idx].get('children', set())
            siblings |= individuals[idx].get('siblings', set())
        individuals[idx] = {'sex': v.get_gender().lower(), 'children': children, 'siblings': siblings, 'title': title}

    elif isinstance(v,FamilyElement): # jeśli wartość elementu to rodzina
        wife, husb, children = None, None, set()
        children = set(term2id(i) for i in g.get_family_members(v, "CHIL")) # dodaj dzieci

        # wyekstrahuj informację o żonie (sprawdź czy jest w nowym słowniku i dodaj odpowiednie info o dzieciach)
        try:
            wife = g.get_family_members(v, "WIFE")[0]
            wife = term2id(wife)
            if wife in individuals:
                individuals[wife]['children'] |= children
            else:
                individuals[wife] = {'children': children}
        except IndexError:
            pass

        # wyekstrahuj informację o mężu (sprawdź czy jest w nowym słowniku i dodaj odpowiednie info o dzieciach)
        try:
            husb = g.get_family_members(v, "HUSB")[0]
            husb = term2id(husb)
            if husb in individuals:
                individuals[husb]['children'] |= children
            else:
                individuals[husb] = {'children': children}
        except IndexError:
            pass

        # wykestrahuj informację o małżeństwie
        if wife and husb:
            marriages[wife + husb] = (term2id(v), wife, husb)

# dla każdej osoby w nowym słowniku
for idx, val in individuals.items():
    added_terms = ''
    if val['sex'] == 'f':
        parent_predicate, sibl_predicate = "isMotherOf", "isSisterOf" # dodaj predykaty charakterystyczne dla kobiet
    else:
        parent_predicate, sibl_predicate = "isFatherOf", "isBrotherOf" # dodaj predykaty charakterystyczne dla mężczyzn
    # dodaj predykaty związane z posiadaniem dzieci
    if len(val['children']):
        added_terms += " ;\n    fhkb:" + parent_predicate + " " + ", ".join(["fhkb:" + i for i in val['children']])
    # dodaj predykaty związane z posiadaniem rodzeństwa
    if len(val['siblings']):
        added_terms += " ;\n    fhkb:" + sibl_predicate + " " + ", ".join(["fhkb:" + i for i in val['siblings']])
    out.write("fhkb:%s a owl:NamedIndividual, owl:Thing%s ;\n    rdfs:label \"%s\" .\n" % (idx, added_terms, val['title']))

for k, v in marriages.items():
    out.write("fhkb:%s a owl:NamedIndividual, owl:Thing ;\n    fhkb:hasFemalePartner fhkb:%s ;\n    fhkb:hasMalePartner fhkb:%s .\n" % v)

out.write("[] a owl:AllDifferent ;\n    owl:distinctMembers (")
for idx in individuals.keys():
    out.write("    fhkb:" + idx)
for k, v in marriages.items():
    out.write("    fhkb:" + v[0])
out.write("    ) .")
out.close()

**Zadanie 2**

Powyższa pętla nie jest kompletna, należy do niej dodać parę elementów (oznaczone są one kropkami ...). Dopasuje poniższe elementy w odpowiednie miejsca:

    1. "isMotherOf", "isSisterOf"
    2. FamilyElement
    3. "isFatherOf", "isBrotherOf"
    4. IndividualElement
    5. member.get_pointer() == v.get_pointer()

### Wnioskowanie

Teraz chcemy wkorzystać ontologię do wnioskowania i wyszukiwań. Użyjemy [RDFLib](https://github.com/RDFLib), bibilioteki do czytania grafów RDF, przeszukiwania ich itd.

Do wnioskowania, użyjemy biblioteki [OWL-RL](https://github.com/RDFLib/OWL-RL), która pozwala zbudować **domknięcie** grafu RDF (najkrótszą relację przechodznią między dwoma węzłami), czyli dodać wszystkie możliwe powiązania które mogą być wywnioskowane.

In [ ]:
!{sys.executable} -m pip install rdflib
!{sys.executable} -m pip install git+https://github.com/RDFLib/OWL-RL.git

Otwórzmy plik z ontologią i sprawdźmy ile trójek zawiera:

In [ ]:
import rdflib
from owlrl import DeductiveClosure, OWLRL_Extension

g = rdflib.Graph()
g.parse("onto.ttl", format="turtle")

print("Triplets found:%d" % len(g))

Zbudujmy domknięcie grafu i sprawdźmy jak liczba trójek wzrosła:

In [ ]:
DeductiveClosure(OWLRL_Extension).expand(g)
print("Triplets after inference:%d" % len(g))

### Poszukiwania krewnych

Możemy przeszukać graf by zobaczyć różne powiązania między ludźmi. Możemy użyć języka **SPARQL** razem z metodą `query`. Zobaczmy wszystkich **wujów** w naszym rodzinnym drzewie:

In [ ]:
qres = g.query(
    """SELECT DISTINCT ?aname ?bname
       WHERE {
          ?a fhkb:isUncleOf ?b .
          ?a rdfs:label ?aname .
          ?b rdfs:label ?bname .
       }""")

for row in qres:
    print("%s is uncle of %s" % row)

**Zadanie 3**

Sprawdź za pomocą zapytania SPARQL wszystkich **przodków** w rodzinie (przejrzyj plik z ontologią by znaleźć odpowiednią nazwę relacji)

In [ ]:
qres = g.query(
    """SELECT DISTINCT ?aname ?bname
       WHERE {
          ?a fhkb:isAncestorOf ?b .
          ?a rdfs:label ?aname .
          ?b rdfs:label ?bname .
       }""")

for row in qres:
    print("%s is ancestor of %s" % row)